In [1]:
import numpy as np 
import tensorflow as tf 
from tensorflow.keras import layers,models
import matplotlib.pyplot as plt


**CHECK GPU** (OPTIONAL)

In [2]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("CUDA version:", tf.sysconfig.get_build_info()["cuda_version"])
print("cuDNN version:", tf.sysconfig.get_build_info()["cudnn_version"])

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

from tensorflow.python.client import device_lib
print("LOCAL DEVICE : ",device_lib.list_local_devices())



TensorFlow version: 2.10.0
CUDA version: 64_112
cuDNN version: 64_8
Num GPUs Available:  1
Num CPUs Available:  1
LOCAL DEVICE :  [name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 7809577467553328877
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 2238133044
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1901722397547598531
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1650, pci bus id: 0000:01:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


**CODEE**

In [3]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
EPOCHS = 50
CHANNELS = 3

In [4]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
  'PlantVillage/',
  shuffle = True,
  image_size = (256,256),
  batch_size = BATCH_SIZE
)

In [ ]:
dataset.class_names

: 

In [ ]:
class_names = dataset.class_names
class_names

: 

In [ ]:
len(dataset)

: 

In [ ]:
dataset.take(1)

: 

In [ ]:
for image_batch,label_batch in dataset.take(1):
  print(image_batch.shape)
  print(label_batch.numpy().astype(np.int32))

: 

In [ ]:
plt.figure(figsize=(10,10))
for image_batch,label_batch in dataset.take(1):
  for i in range(12):
    ax = plt.subplot(3,4,i+1)
    plt.imshow(image_batch[i].numpy().astype(np.int32))
    plt.title(class_names[label_batch[i]])
    plt.axis("off")
    


: 

In [ ]:
print(image_batch[i].shape)

: 

80 % -- > TRAINING  
10 % -- > VALIDATION  
10 % -- > TESTING

In [ ]:
train_size = 0.8
len(dataset) * train_size

: 

In [ ]:
train_ds = dataset.take(54)
len(train_ds)

: 

In [ ]:
test_ds_1 = dataset.skip(54)
len(test_ds_1)

: 

In [ ]:
val_size = 0.1
len(dataset) * val_size

: 

In [ ]:
val_ds = test_ds_1.take(6)
len(val_ds)

: 

In [ ]:
test_ds = test_ds_1.skip(6)
len(test_ds)

: 

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1
    
    ds_size = len(ds)
    
    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)
    
    train_ds = ds.take(train_size)    
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)
    
    return train_ds, val_ds, test_ds

: 

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

: 

In [ ]:
[len(train_ds), len(val_ds), len(test_ds)]

: 

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

: 

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory


: 

In [ ]:
resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
  layers.Rescaling(1.0/255),
])

: 

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

: 

In [ ]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    resize_and_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(n_classes, activation='softmax'),
])

model.build(input_shape=input_shape)

: 

In [ ]:
model.summary()

: 

In [ ]:
model.compile(
  optimizer = 'adam',
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
  metrics = ['accuracy']
)

: 

In [ ]:
tf.debugging.set_log_device_placement(True)


: 

In [ ]:
import tensorflow as tf

# Check if any GPU devices are being used
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

if tf.config.experimental.list_physical_devices('GPU'):
    print("GPU is being used for training.")
else:
    print("Training is being performed on the CPU.")

: 

In [ ]:
history = model.fit(
  train_ds,
  epochs = 20,
  batch_size = BATCH_SIZE,
  verbose = 1,
  validation_data = val_ds,
)
  
  

: 

In [ ]:
scores = model.evaluate(test_ds)

: 

In [ ]:
scores

: 

In [ ]:
history

: 

In [ ]:
history.params

: 

In [ ]:
history.history.keys()

: 

In [ ]:
history.history['accuracy']

: 

In [ ]:
acc= history.history['accuracy']
val_acc= history.history['val_accuracy']
loss= history.history['loss']
val_loss= history.history['val_loss']

: 

**PLOTTING**

In [ ]:
n = len(acc)

: 

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(n),acc,label='Training Accuracy')
plt.plot(range(n),val_acc,label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1,2,2)
plt.plot(range(n),loss,label='Training Loss')
plt.plot(range(n),val_loss,label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')

: 

In [ ]:

import numpy as np
for images_batch, labels_batch in test_ds.take(1):
    
    first_image = images_batch[0].numpy().astype('uint8')
    first_label = labels_batch[0].numpy()
    
    print("first image to predict")
    plt.imshow(first_image)
    print("actual label:",class_names[first_label])
    
    batch_prediction = model.predict(images_batch)
    print("predicted label:",class_names[np.argmax(batch_prediction[0])])

: 

**Function for inference**

In [ ]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img[i].numpy())
    img_array = tf.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

: 

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]] 
        
        plt.title(f"Actual: {actual_class},\n Predicted: {predicted_class}.\n Confidence: {confidence}%")
        
        plt.axis("off")

: 

**Saving the Model**


In [ ]:
# import os
# model_version = max([int(i) for i in  os.listdir("models") + [0]]) + 1
# model.save(f"models/{model_version}")

: 

In [ ]:
model_version = 2
model.save(f"models/{model_version}")

: 